## run request with owslib

In [1]:
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput

In [2]:
wps_url = "http://localhost:8093/wps"
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [3]:
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract, )

visualisation : 	 Just testing a nice script to visualise some variables
extractpoints : 	 Extract Timeseries for specified coordinates from grid data
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
indices_single : 	 This process calculates climate indices based on one single variable.
subset_countries : 	 This process returns only the given polygon from input netCDF files.
eobs_to_cordex : 	 downloads EOBS data in adaped CORDEX format
ensembleRobustness : 	 Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries
analogs : 	 Search for days with analog pressure pattern


In [9]:
p = wps.describeprocess(identifier='subset_countries')
for input in p.dataInputs:
    printInputOutput(input)
    print '\n'

 identifier=resource, title=Resource, abstract=NetCDF File, data type=ComplexData
 Supported Value: mimeType=application/x-netcdf, encoding=None, schema=None
 Default Value: mimeType=application/x-netcdf, encoding=None, schema=None 
 minOccurs=1, maxOccurs=1000


 identifier=region, title=Region, abstract=None, data type=//www.w3.org/TR/xmlschema-2/#string
 Allowed Value: ABW
 Allowed Value: AFG
 Allowed Value: AGO
 Allowed Value: ALB
 Allowed Value: ALD
 Allowed Value: AND
 Allowed Value: ARE
 Allowed Value: ARG
 Allowed Value: ARM
 Allowed Value: ASM
 Allowed Value: ATA
 Allowed Value: ATF
 Allowed Value: ATG
 Allowed Value: AUS
 Allowed Value: AUT
 Allowed Value: AZE
 Allowed Value: BDI
 Allowed Value: BEL
 Allowed Value: BEN
 Allowed Value: BFA
 Allowed Value: BGD
 Allowed Value: BGR
 Allowed Value: BHR
 Allowed Value: BHS
 Allowed Value: BIH
 Allowed Value: BLR
 Allowed Value: BLZ
 Allowed Value: BOL
 Allowed Value: BRA
 Allowed Value: BRB
 Allowed Value: BRN
 Allowed Value: BTN
 

In [4]:
in_file1 = 'file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p1_19600101-20131230.nc'
in_file2 = 'file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p2_19600101-20131230.nc'

#in_file1 = 'file:///home/pingu/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tasmax/v20150609/tasmax_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_199101-200012.nc'
#in_file2 = 'file:///home/pingu/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tasmax/v20150609/tasmax_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_200101-200512.nc'

In [6]:
execute = wps.execute(
    identifier="subset_countries", #indices_clipping",
    inputs=[
    ("resource",in_file1),
    ("resource",in_file2),
    ("region",'POL'),
    ("region",'DEU'),
    ("mosaik",'True'),  
           ])

monitorExecution(execute, sleepSecs=5)

print execute.getStatus()

for o in execute.processOutputs:
    print o.reference
    

ProcessSucceeded
http://localhost:8090/wpsoutputs/flyingpigeon/output-5a18a28e-d405-11e5-8a28-9d33cb3eae7f.tar


In [4]:
# for rotated grids 

rot_file1 = 'file:///home/starmip/tnoel/BCCORDEX/TasAdjust/tasAdjust_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_SMHI-RCA4-LSCE-IPSL-CDFt-EOBS10-1971-2005_day_20860101-20901231.nc' 
rot_file2 = 'file:///home/starmip/tnoel/BCCORDEX/TasAdjust/tasAdjust_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_SMHI-RCA4-LSCE-IPSL-CDFt-EOBS10-1971-2005_day_20910101-20951231.nc'
rot_file3 = 'file:///home/starmip/tnoel/BCCORDEX/TasAdjust/tasAdjust_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_SMHI-RCA4-LSCE-IPSL-CDFt-EOBS10-1971-2005_day_20960101-21001231.nc'

In [5]:
# define a dimensinon_map: 

dimension_map = "{'X': {'variable': 'lon', 'dimension': 'x', 'pos': 2}, 'Y': {'variable': 'lat', 'dimension': 'y', 'pos': 1}, 'T': {'variable': 'time', 'dimension': 'time', 'pos': 0 }}"


In [ ]:
execute = wps.execute(
    identifier="subset_countries", #indices_clipping",
    inputs=[
    ("resource",rot_file1),
   # ("resource",rot_file2),
   # ("resource",rot_file3),
   # ("region",'POL'),
    ("region",'DEU'),
   # ("mosaik",'True'),
    ("dimension_map", dimension_map),
    ("variable",'tasAdjust')
           ])

monitorExecution(execute, sleepSecs=5)

print execute.getStatus()

for o in execute.processOutputs:
    print o.reference
    

## run sync request with get parameters

In [ ]:
sync_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=subset_countries" +\
    "&DataInputs=resource={resource1};resource={resource2};region={region}" +\
    "&storeExecuteResponse=false" +\
    "&status=false"

In [ ]:
url=sync_req_url.format(
    wps_url=wps_url,
    resource1=in_file1,
    resource2=in_file2,
    region='FRA')
print url   

In [ ]:
import requests
r = requests.get(url)
print r.status_code
print r.text

## run async request with get parameters

In [ ]:
async_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=subset_countries" +\
    "&DataInputs=resource={resource1};resource={resource2};region={region}" +\
    "&storeExecuteResponse=true" +\
    "&status=true"
    #

In [ ]:
url=async_req_url.format(
    wps_url=wps_url,
    resource1=in_file1,
    resource2=in_file2,
    region='FRA')
print url   

In [ ]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
#print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

In [ ]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
#print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

In [ ]:
r = requests.get(status_url)
print r.status_code
print r.text

In [ ]:
status_url2 = status_url

In [ ]:
r = requests.get(status_url2)
print r.status_code
print r.text